In [56]:
import re
import pandas as pd
import numpy as np
import datetime

In [57]:
def parser(file):
    #Initialize list
    lines = []
    
    #Open File
    with open(file, 'r') as file:
        for line in file:
            lines.append(line.rstrip('\n'))

    #Extract Date
    Date = (re.split('[\s]{4,}', lines[6].strip()))
    Date = Date[0]
    Date = Date.replace('DATE: ', '')
    Date

    #Extract indices to cut list
    indices = [i for i, s in enumerate(lines) if '------' in s]

    #Create list with only well licenses
    df1 = lines[(indices[1]+2):(indices[2]-1)]

    #Create indexes to slice
    length = len(df1)
    times = length//5
    index3 = np.arange(2, length, 6).tolist()
    index4 = np.arange(4, length, 6).tolist()
    indexr = np.arange(0, length, 1).tolist()
    index = index3 + index4
    indexr = np.delete(indexr, index)

    #Create 3 lists to bypass issue with 3rd line and 5th line  
    dfline3 = []
    dfline4 = []
    dfliner = []
    for line in index3:
        dfline3.append(df1[line])

    for line in index4:
        dfline4.append(df1[line])    

    for line in indexr:
        dfliner.append(df1[line])

    df2 = []
    for line in dfliner:
        df2.append(line[0:40])
        df2.append(line[41:50])
        df2.append(line[51:71])
        df2.append(line[72:104])

    df3 = []
    for line in dfline3:
        df3.append(line[0:40])
        df3.append(line[41:71])
        df3.append(line[72:104])

    dfl = []
    for line in dfline4:
        dfl.append(line[0:60])
        dfl.append(line[72:104])    

    #Create a list without all spaces
    df4 = []
    for line in df2:
        df4.extend(re.split('[\s]{5,}', line.strip()))

    #Create list without blank items
    df4 = [x for x in df4 if x != '']

    #Create lists for all 17 elements in the table
    df4 = [df4[x:x+12] for x in range(0, len(df4), 12)]

    #Create list for 3rd line
    dfr1 = []
    for line in df3:
        dfr1.extend(re.split('[\s]{4,}', line.strip()))

    #Create lists for all 17 elements in the table
    dfr1 = [dfr1[x:x+3] for x in range(0, len(dfr1), 3)]

    #Create list for 5th line
    dfr2 = []
    for line in dfl:
        dfr2.extend(re.split('[\s]{4,}', line.strip()))

    #Create lists for all 17 elements in the table
    dfr2 = [dfr2[x:x+2] for x in range(0, len(dfr2), 2)]

    #create a dataframe out of the list
    if df4 != []:
        data1 = pd.DataFrame(df4)
        data1.set_axis(['WELL NAME', 'LICENCE NUMBER', 'MINERAL RIGHTS', 'GROUND ELEVATION', 'UID', 'SURFACE CO-ORDINATES', 'BOARD FIELD CENTRE', 'PROJECTED DEPTH', 'DRILLING OPERATION', 'WELL PURPOSE', 'WELL TYPE', 'SUBSTANCE'], axis = 1, inplace=True)
        data1['DATE'] = Date

        data2 = pd.DataFrame(dfr1)
        data2.set_axis(['LAHEE CLASSIFICATION', 'FIELD', 'TERMINATING ZONE'],axis = 1, inplace=True)

        data3 = pd.DataFrame(dfr2)
        data3.set_axis(['LICENSEE', 'SURFACE LOCATION'],axis = 1, inplace=True)

        datac = pd.concat([data1, data2, data3], axis=1, sort=False)
        
        #Conversion of Surface Location to numbers only in order to search ATS values and match them to latitude and longitude
        datac['ATS'] = datac['SURFACE LOCATION'].str.replace(r'[^0-9]+', '')
        datac['ATS'] = datac['ATS'].str[-8:]
        datac['PROJECTED DEPTH'] = datac['PROJECTED DEPTH'].str[:-1]
        
        #Conversion to integers
        datac['ATS'] = datac['ATS'].astype('int64')
        datac['PROJECTED DEPTH'] = datac['PROJECTED DEPTH'].astype('float64')

        datac['DATE'] = datac['DATE'].astype('datetime64')
        datac['WEEK'] = datac['DATE'].dt.week
        datac['MONTH'] = datac['DATE'].dt.month
        datac['YEAR'] = datac['DATE'].dt.year

        convcomp = pd.read_csv('ConvComp.csv')
        convcomp['ConvDF'] = convcomp['ConvDF'].astype('int64')
        dfm = pd.merge(datac, convcomp, how='left', left_on='ATS', right_on='ConvDF')

        return dfm

In [58]:
import os

from pathlib import Path

directory_in_str = "Data/ST1Hist/Unpacked/"

directory = os.fsencode(directory_in_str) 

columns = ['WELL NAME', 'LICENCE NUMBER', 'MINERAL RIGHTS', 'GROUND ELEVATION', 'UID', 'SURFACE CO-ORDINATES', 'BOARD FIELD CENTRE', 'PROJECTED DEPTH', 'DRILLING OPERATION', 'WELL PURPOSE', 'WELL TYPE', 'SUBSTANCE', 'DATE', 'LAHEE CLASSIFICATION', 'FIELD', 'TERMINATING ZONE', 'LICENSEE', 'SURFACE LOCATION']

df = pd.DataFrame(columns=columns)

ext = ['.txt','.TXT']

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    if filename.endswith(tuple(ext)):
        print(filename) 
        data = parser(os.path.join(directory_in_str, filename))
        df = df.append(data, ignore_index=True, sort=False)
        continue 
    else: 
        continue

5729.TXT
31161.txt
30611.TXT
5212.TXT
23258.TXT
3537.TXT
30118.TXT
29516.TXT
13579.TXT
13394.TXT
1538.TXT
613.TXT
6813.TXT
2911.TXT
30154.TXT
29528.TXT
5461.TXT
31981.TXT
21923.TXT
29511.TXT
25052.TXT
17270.txt
22362.TXT
10759.TXT
21534.TXT
2953.TXT
21702.TXT
29343.TXT
11571.TXT
29279.TXT
29172.TXT
29060.TXT
9849.TXT
4863.TXT
29448.TXT
11503.TXT
7940.TXT
87.TXT
11830.TXT
14368.TXT
6153.TXT
19495.TXT
11520.TXT
31302.TXT
10388.TXT
25313.TXT
18473.TXT
16457.TXT
25390.TXT
10911.txt
32468.TXT
8249.TXT
21242.TXT
12816.TXT
1188.TXT
23338.TXT
6297.TXT
12809.TXT
23920.TXT
31292.TXT
25274.TXT
3262.TXT
29406.TXT
9065.TXT
13052.TXT
27905.TXT
6388.TXT
23172.TXT
28703.TXT
28701.txt
28014.txt
21160.TXT
10618.TXT
5923.TXT
22183.TXT
20712.TXT
6364.txt
4692.TXT
26533.TXT
14503.TXT
14331.TXT
20856.TXT
2693.txt
2064.TXT
19213.TXT
6500.TXT
10977.TXT
19867.TXT
4764.TXT
24957.TXT
17310.TXT
11437.TXT
12488.TXT
11792.TXT
27219.txt
3125.TXT
26032.TXT
27724.TXT
18517.TXT
17559.TXT
22511.TXT
18700.TXT
14798.TXT
1

In [59]:
df

,WELL NAME,LICENCE NUMBER,MINERAL RIGHTS,GROUND ELEVATION,UID,SURFACE CO-ORDINATES,BOARD FIELD CENTRE,PROJECTED DEPTH,DRILLING OPERATION,WELL PURPOSE,...,TERMINATING ZONE,LICENSEE,SURFACE LOCATION,ATS,WEEK,MONTH,YEAR,ConvDF,Lat,Long
0,RSO 07 HZ PROVOST 6-19-40-6,0479796,FREEHOLD,679.6M,107/06-19-040-06W4/00 N 361.2M W,691.4M,WAINWRIGHT,1321.0,HORIZONTAL,NEW,...,DINA MBR,RIFLE SHOT OIL CORP.,02-19-040-06W4,19040064.0,26.0,6.0,2016.0,19040064.0,52.455742,-110.844801
1,LONG RUN HZ 102 GVILLEE 1-33-76-21,0479797,ALBERTA CROWN,578.3M,102/01-33-076-21W5/00 N 672.1M W,70.2M,GRANDE PRAIRIE,2500.0,HORIZONTAL,NEW,...,MONTNEY FM,LONG RUN EXPLORATION LTD.,08-32-076-21W5,32076215.0,26.0,6.0,2016.0,32076215.0,55.628652,-117.209125
2,LONG RUN HZ GVILLEE 8-33-76-21,0479798,ALBERTA CROWN,578.3M,100/08-33-076-21W5/00 N 692.1M W,70.2M,GRANDE PRAIRIE,2500.0,HORIZONTAL,NEW,...,MONTNEY FM,LONG RUN EXPLORATION LTD.,08-32-076-21W5,32076215.0,26.0,6.0,2016.0,32076215.0,55.628652,-117.209125
3,LONG RUN HZ 102 ROXANA 13-14-78-21,0479799,ALBERTA CROWN,599.4M,102/13-14-078-21W5/00 S 430.0M E,192.8M,GRANDE PRAIRIE,2500.0,HORIZONTAL,NEW,...,MONTNEY FM,LONG RUN EXPLORATION LTD.,12-13-078-21W5,13078215.0,26.0,6.0,2016.0,13078215.0,55.759612,-117.105627
4,LONG RUN HZ 102 ROXANA 12-14-78-21,0479800,ALBERTA CROWN,599.3M,102/12-14-078-21W5/00 S 465.0M E,192.8M,GRANDE PRAIRIE,2500.0,HORIZONTAL,NEW,...,MONTNEY FM,LONG RUN EXPLORATION LTD.,12-13-078-21W5,13078215.0,26.0,6.0,2016.0,13078215.0,55.759612,-117.105627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35097,BIR 6-32 HZ 102 PCOUPE 15-4-79-12,0491534,ALBERTA CROWN,801M,102/15-04-079-12W6/00 N 401.0M E,619.3M,GRANDE PRAIRIE,4900.0,HORIZONTAL,NEW,...,MONTNEY FM,BIRCHCLIFF ENERGY LTD.,06-32-078-12W6,32078126.0,43.0,10.0,2018.0,32078126.0,55.803123,-119.810472
35098,TOURMALINE HZ VALHALLA 4-21-77-8,0491535,ALBERTA CROWN,965.2M,100/04-21-077-08W6/00 N 426.2M E,520.8M,GRANDE PRAIRIE,4400.0,HORIZONTAL,NEW,...,MONTNEY FM,TOURMALINE OIL CORP.,06-28-077-08W6,28077086.0,43.0,10.0,2018.0,28077086.0,55.701390,-119.164027
35099,TXNE 8A-14 HZ WAYNE 13-1-27-19,0491536,FREEHOLD,862.3M,102/13-01-027-19W4/00 N 429.4M W,150.1M,MIDNAPORE,3957.0,HORIZONTAL,NEW,...,BASAL QUARTZ SD,TORXEN ENERGY LTD.,08-14-027-19W4,14027194.0,43.0,10.0,2018.0,14027194.0,51.306978,-112.559913
35100,TXNE 8A-14 HZ WAYNE 15-2-27-19,0491537,BOTH,862.7M,100/15-02-027-19W4/00 N 414.4M W,150.1M,MIDNAPORE,3900.0,HORIZONTAL,NEW,...,BASAL QUARTZ SD,TORXEN ENERGY LTD.,08-14-027-19W4,14027194.0,43.0,10.0,2018.0,14027194.0,51.306978,-112.559913


Merging ATS with Coordinate CSV to determine latitute and longitude

Export to CSV

In [60]:
df.to_csv(r'ST1 Historical.csv', index = False)